In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from sklearn import linear_model, datasets

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((15*21,3), np.float32)
objp[:,:2] = np.mgrid[0:21,0:15].T.reshape(-1,2)

In [3]:
objp.shape

(315, 3)

In [4]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('a.png')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (21,15),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (21,15), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [5]:
print(corners.shape)

(315, 1, 2)


In [6]:
imgp = np.ones((3,315))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,315))
imgp[:2,:] = imgp1
print(imgp.shape)

objp = objp.T
print(objp.shape)
obj = np.ones((4,315))
obj[:3,:] = objp

(3, 315)
(3, 315)


In [7]:
obj

array([[ 0.,  1.,  2., ..., 18., 19., 20.],
       [ 0.,  0.,  0., ..., 14., 14., 14.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [8]:
imgp

array([[107.56755066,  83.28165436, 130.83979797,  83.35715485,
        154.64964294,  83.52548218, 178.43469238,  83.59564972,
        202.1711731 ,  84.16691589, 225.64862061,  84.19496155,
        249.3981781 ,  84.3937149 , 273.0770874 ,  84.56157684,
        296.48498535,  84.66824341, 319.58239746,  84.62976837,
        343.1633606 ,  84.49532318, 366.5144043 ,  84.71075439,
        389.93441772,  84.65130615, 413.60021973,  84.61601257,
        437.34890747,  84.27425385, 461.1373291 ,  84.37445831,
        485.06295776,  84.3088913 , 509.17733765,  84.09620667,
        533.06195068,  83.88614655, 557.16821289,  83.61513519,
        580.50585938,  83.75165558, 107.33692169, 106.62995148,
        130.93611145, 106.88222504, 154.7782135 , 107.22281647,
        178.62882996, 107.39172363, 202.2800293 , 107.50106812,
        225.58125305, 107.51467896, 249.2624054 , 107.59059143,
        272.77856445, 107.61261749, 296.31332397, 107.84807587,
        319.44625854, 108.12561798, 342.

We now have the image and object points

Arranging the matrix in "THAT" form

In [10]:
A = []

In [11]:
for i in range(42):
    A.append(np.array([obj[0,i], obj[1][i], obj[2][i],1,0,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i], (-1)*obj[2][i]*imgp[0,i],(-1)*imgp[0,i]]))
    A.append(np.array([0,0,0,0, obj[0,i], obj[1][i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i], (-1)*obj[2][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [12]:
A

[array([   0.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
          -0.        ,   -0.        ,   -0.        , -107.56755066]),
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    1.        ,
          -0.        ,   -0.        ,   -0.        , -248.49008179]),
 array([  1.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
        -83.28165436,  -0.        ,  -0.        , -83.28165436]),
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           1.        ,    0.        ,    0.        ,    1.        ,
        -365.20776367,   -0.        ,   -0.        , -365.20776367]),
 array([   2.        ,    0.        ,    0.        ,    1.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
        -261.67959595,   -0.        ,   -0.        ,

In [13]:
len(A)

84

In [14]:
a = np.array(A)
# d.reshape((12,12))
a.shape

(84, 12)

In [15]:
ata = np.matmul(a.T,a)

In [16]:
u1, s1, vh1 = np.linalg.svd(ata, full_matrices=True)

print(u1.shape)
print(s1.shape)
print(vh1.shape)

(12, 12)
(12,)
(12, 12)


In [17]:
s1

array([1.24912958e+09, 2.64359278e+06, 9.98253234e+05, 5.64967700e+03,
       1.24004962e+03, 1.74302347e+01, 6.25332185e+00, 4.12795805e+00,
       8.85566184e-01, 3.13222102e-11, 5.28265592e-17, 0.00000000e+00])

In [18]:
vh1[:,8]

array([ 9.97011161e-01, -6.53195048e-02, -4.12226645e-02, -5.07631652e-05,
       -1.66043598e-03,  3.99225632e-05,  7.43977026e-05,  1.65125826e-04,
        7.14457458e-05, -1.75151998e-17,  4.06781638e-20,  0.00000000e+00])

In [19]:
p1 = vh1[:,8]

In [20]:
p1.reshape((3,4))

array([[ 9.97011161e-01, -6.53195048e-02, -4.12226645e-02,
        -5.07631652e-05],
       [-1.66043598e-03,  3.99225632e-05,  7.43977026e-05,
         1.65125826e-04],
       [ 7.14457458e-05, -1.75151998e-17,  4.06781638e-20,
         0.00000000e+00]])

Now trying the same things after excluding the Z

In [22]:
obj

array([[ 0.,  1.,  2., ..., 18., 19., 20.],
       [ 0.,  0.,  0., ..., 14., 14., 14.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [21]:
obj.shape

(4, 315)

In [23]:
C = []

In [24]:
for i in range(315):
    C.append(np.array([obj[0,i], obj[1][i],1,0,0,0, (-1)*obj[0,i]*imgp[0,i], (-1)*obj[1][i]*imgp[0,i],(-1)*imgp[0,i]]))
    C.append(np.array([0,0,0, obj[0,i], obj[2][i],1, (-1)*obj[0,i]*imgp[1,i], (-1)*obj[1][i]*imgp[1,i],(-1)*imgp[1,i]]))

In [33]:
C.shape

(630, 9)

In [26]:
len(C)

630

In [27]:
C = np.array(C)
C.shape

(630, 9)

In [34]:
ctc = np.matmul(C.T,C)
# ctcinv = inv(ctc)
# ctcinv.shape

In [35]:
print(ctc.shape)
# print(ctc)

(9, 9)


In [36]:
u, s, vh = np.linalg.svd(ctc, full_matrices=True)

In [37]:
print(u.shape)
print(s.shape)
print(vh.shape)

(9, 9)
(9,)
(9, 9)


In [38]:
s

array([1.42525961e+10, 1.56351572e+09, 8.18258329e+06, 5.17699245e+04,
       7.24210744e+03, 4.75993822e+03, 5.12241340e+01, 1.22852460e+01,
       1.38281312e-13])

In [39]:
vh[:,8]

array([ 6.33120467e-02, -2.22864650e-02,  9.97732759e-01,  6.46780327e-05,
       -4.02900363e-03,  1.59322630e-03, -2.72250249e-04, -2.32176950e-03,
       -1.52989045e-17])

In [40]:
p = vh[:,8]

In [41]:
p = p.reshape((3,3))
# pinv = inv(p)
# pinv

array([[-3.69772654e+00,  2.31564273e+03, -3.98287760e+03],
       [ 4.33594711e-01, -2.71531826e+02,  3.63254919e+01],
       [ 1.24660027e+00, -1.53006468e+02,  2.53548555e+02]])

We need to normalize the matrix H

In [47]:
from numpy import linalg as LA

norm = LA.norm(p)
p = p/norm
p

array([[ 6.33120467e-02, -2.22864650e-02,  9.97732759e-01],
       [ 6.46780327e-05, -4.02900363e-03,  1.59322630e-03],
       [-2.72250249e-04, -2.32176950e-03, -1.52989045e-17]])

In [49]:
pinv = inv(p)

norm = LA.norm(pinv)
pinv = pinv/norm
pinv

array([[-7.99552315e-04,  5.00706985e-01, -8.61209980e-01],
       [ 9.37553523e-05, -5.87128057e-02,  7.85459142e-03],
       [ 2.69549985e-04, -3.30842950e-02,  5.48243175e-02]])

In [50]:
check = np.matmul(pinv,imgp)
check

array([[123.47350375, 181.93428017, 123.46135232, 193.69348409,
        123.50725666, 205.50649236, 123.54503638, 217.19426484,
        123.57561968, 229.13842319, 123.64555839, 241.10535577,
        123.660912  , 253.13626827, 123.75474092, 265.12420652,
        123.73746144, 277.10475216, 123.79902476, 288.89068089,
        123.77983438,  52.97767976, 134.20471249,  64.65025055,
        134.28301728,  76.30373821, 134.38123451,  88.23505706,
        134.38539595,  99.96066455, 134.40537135, 111.80496035,
        134.48637379, 123.63248097, 134.56282456, 135.45901139,
        134.50405974, 146.97316323, 134.57446598, 158.63332183,
        134.57593055, 170.28239722, 135.02619382, 181.93257319,
        135.10799208, 193.64003398, 135.26300298, 205.52881736,
        135.38122724, 217.14229417, 135.40523942, 229.13449659,
        135.44542299, 241.07508208, 135.45219025, 253.09973616,
        135.55083273, 264.98909514, 135.57752038, 277.0805955 ,
        135.61553118, 288.82146953, 135.